In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit

### Import predicted data.

In [2]:
# close_predict = pd.read_csv("SCINet_test_res.csv", index_col = [0])
# close_predict = pd.read_csv("Transformer_test_res.csv", index_col = [0])
# close_predict = pd.read_csv("Actual_test_res.csv", index_col = [0])
close_predict = pd.read_csv("TCN_test_res.csv", index_col = [0])
close_predict = close_predict.to_dict('list')
for the_key in close_predict:
    close_predict[the_key] =  np.array(close_predict[the_key])
close_predict

{'2021-05-18': array([126.43231 , 126.665474, 126.50596 , 126.83819 , 127.32628 ,
        127.27823 , 127.26253 , 127.495705, 127.55394 , 126.633575,
        128.74533 , 128.6214  , 128.68338 , 128.15628 , 128.18265 ]),
 '2021-05-19': array([124.59109 , 124.85593 , 125.04015 , 126.084625, 126.13777 ,
        126.11645 , 126.00035 , 126.31178 , 126.24575 , 125.35769 ,
        127.12998 , 126.93393 , 126.87971 , 126.75367 , 127.17401 ]),
 '2021-05-20': array([124.85554 , 124.85742 , 125.08242 , 125.914474, 125.65587 ,
        125.52498 , 125.79268 , 126.143196, 126.22796 , 125.17253 ,
        126.864105, 126.50407 , 127.17207 , 126.68672 , 126.85139 ]),
 '2021-05-21': array([127.038994, 127.67906 , 127.554855, 128.51358 , 128.21709 ,
        128.34308 , 128.2825  , 128.94356 , 128.95108 , 127.72036 ,
        129.47684 , 129.06985 , 129.30527 , 129.68585 , 129.31464 ]),
 '2021-05-24': array([125.17418 , 125.731285, 126.030235, 126.29776 , 126.137505,
        126.380554, 126.560684, 126.91

In [3]:
close_true = pd.read_csv("Actual_test_res.csv", index_col = [0])
close_true = close_true.to_dict('list')
for the_key in close_true:
    close_true[the_key] =  np.array(close_true[the_key])
close_true

{'2021-05-18': array([124.84999 , 124.689995, 127.30999 , 125.42999 , 127.09999 ,
        126.899994, 126.84999 , 125.28    , 124.60999 , 124.27999 ,
        125.06    , 123.54    , 125.89    , 125.899994, 126.73999 ]),
 '2021-05-19': array([124.689995, 127.30999 , 125.42999 , 127.09999 , 126.899994,
        126.84999 , 125.28    , 124.60999 , 124.27999 , 125.06    ,
        123.54    , 125.89    , 125.899994, 126.73999 , 127.12999 ]),
 '2021-05-20': array([127.30999 , 125.42999 , 127.09999 , 126.899994, 126.84999 ,
        125.28    , 124.60999 , 124.27999 , 125.06    , 123.54    ,
        125.89    , 125.899994, 126.73999 , 127.12999 , 126.11    ]),
 '2021-05-21': array([125.42999 , 127.09999 , 126.899994, 126.84999 , 125.28    ,
        124.60999 , 124.27999 , 125.06    , 123.54    , 125.89    ,
        125.899994, 126.73999 , 127.12999 , 126.11    , 127.34999 ]),
 '2021-05-24': array([127.09999 , 126.899994, 126.84999 , 125.28    , 124.60999 ,
        124.27999 , 125.06    , 123.54

### Calculate the profit with initial values

In [4]:
@njit
def calculate_profit(the_close_predict, the_close_true,budget=10000,sell_t=1,buy_t=1,stock_num=0):
#     close_predict=get_close_predict(stock_name)
#     close_true=get_close_true(stock_name)
#     close_predict=get_close_predict(stock_name)
    n=len(the_close_predict)
    delt=np.diff(the_close_predict)
    Delt=np.diff(delt)
    initial_budget=budget
    counter = 0
    for i in range(0,n-2):
        if Delt[i]>0 and stock_num>0:##sell
#             print('sell')
            counter += 1
            sell_num=int(Delt[i])+sell_t
            if stock_num>sell_num:
                budget=budget+the_close_true[i]*sell_num
                stock_num=stock_num-sell_num
            else:
                sell_num=stock_num
                budget=budget+the_close_true[i]*sell_num
                stock_num=stock_num-sell_num
        elif Delt[i]<0 and -int(Delt[i])+buy_t>0:##buy
#             print('buy')
            counter += 1
            buy_num=-int(Delt[i])+buy_t
            if budget>the_close_true[i]*buy_num:
                stock_num=stock_num+buy_num
                budget=budget-the_close_true[i]*buy_num
            else:
                buy_num=math.floor(budget/the_close_true[i])
                stock_num=stock_num+buy_num
                budget=budget-the_close_true[i]*buy_num
#         else:
#             budget=budget
#             stock_num=stock_num
        #print(stock_num)
        #print(budget)
    profit=budget+stock_num*the_close_true[n-1]-initial_budget
    return profit, counter

In [5]:
calculate_profit(close_predict['2021-05-18'], close_true['2021-05-18'], budget=100,sell_t=10,buy_t=10,stock_num=0)

(0.0, 6)

### Find the best sell number and buy number for each time

In [6]:
def find_best(the_close_predict, the_close_true,the_date,budget):
    profit=np.zeros((201,201))
    trading_freq = np.zeros((201,201))
#     for k in range(-100,100):
#         for l in range(-100,100):
    for k in range(0,100):
        for l in range(0,100):
            profit[k,l], trading_freq[k,l] = calculate_profit(the_close_predict, the_close_true, budget,sell_t=k,buy_t=l)
    best_profit=np.max(profit)
    best_sell_t,best_buy_t = np.where(profit==best_profit)
    print(f'For {the_date}, the best sell threshold is {best_sell_t[0]}, the best buy threshold is {best_buy_t[0]} and the best profit is {best_profit}')
    return best_sell_t,best_buy_t,best_profit, trading_freq[best_sell_t[0],best_buy_t[0]]

In [7]:
# stocks=['A','ALLE','AMGN','BIIB','BMY','CAT']
budget=1000
dates = list(close_predict.keys())
budgets= [budget for i in range(len(dates))]
# budgets= [3300,1200,3000,700,1200,700]

In [8]:
best_profit=np.zeros(len(dates))
best_freq = np.zeros(len(dates))

In [9]:
for i, the_date in enumerate(dates):
#     print(find_best(close_predict[the_date], close_true[the_date], the_date, budgets[i]))
    _, _, best_profit[i], best_freq[i] = find_best(close_predict[the_date], close_true[the_date], the_date, budgets[i])

For 2021-05-18, the best sell threshold is 4, the best buy threshold is 8 and the best profit is 25.8700759999997
For 2021-05-19, the best sell threshold is 0, the best buy threshold is 8 and the best profit is 17.959940000000074
For 2021-05-20, the best sell threshold is 8, the best buy threshold is 7 and the best profit is 28.05007599999999
For 2021-05-21, the best sell threshold is 0, the best buy threshold is 1 and the best profit is 9.049952000000076
For 2021-05-24, the best sell threshold is 0, the best buy threshold is 1 and the best profit is 35.74004799999989
For 2021-05-25, the best sell threshold is 8, the best buy threshold is 8 and the best profit is 41.519853999999896
For 2021-05-26, the best sell threshold is 0, the best buy threshold is 2 and the best profit is 44.610040000000026
For 2021-05-27, the best sell threshold is 4, the best buy threshold is 8 and the best profit is 62.63987400000019
For 2021-05-28, the best sell threshold is 0, the best buy threshold is 8 and 

For 2021-09-02, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-09-03, the best sell threshold is 2, the best buy threshold is 2 and the best profit is 3.45975999999996
For 2021-09-07, the best sell threshold is 4, the best buy threshold is 2 and the best profit is 5.230000000000018
For 2021-09-08, the best sell threshold is 1, the best buy threshold is 0 and the best profit is 3.3999999999999773
For 2021-09-09, the best sell threshold is 1, the best buy threshold is 0 and the best profit is 1.979979999999955
For 2021-09-10, the best sell threshold is 1, the best buy threshold is 0 and the best profit is 0.08997999999996864
For 2021-09-13, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-09-14, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-09-15, the best sell threshold is 7, the best buy threshold is 7 and the best profit is 2.329810000000407

For 2021-12-20, the best sell threshold is 1, the best buy threshold is 0 and the best profit is 4.04998999999998
For 2021-12-21, the best sell threshold is 2, the best buy threshold is 5 and the best profit is 20.670029999999883
For 2021-12-22, the best sell threshold is 5, the best buy threshold is 5 and the best profit is 34.29995000000008
For 2021-12-23, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-12-27, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-12-28, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-12-29, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-12-30, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2021-12-31, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-01-03, the best sell thresho

For 2022-04-25, the best sell threshold is 4, the best buy threshold is 2 and the best profit is 7.699940000000083
For 2022-04-26, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-04-27, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-04-28, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-04-29, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-05-02, the best sell threshold is 5, the best buy threshold is 2 and the best profit is 1.7301500000000942
For 2022-05-03, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-05-04, the best sell threshold is 5, the best buy threshold is 3 and the best profit is 18.91987000000006
For 2022-05-05, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-05-06, the best sell thresh

For 2022-08-10, the best sell threshold is 5, the best buy threshold is 2 and the best profit is 3.3399899999999434
For 2022-08-11, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-08-12, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-08-15, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-08-16, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-08-17, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-08-18, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-08-19, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-08-22, the best sell threshold is 1, the best buy threshold is 1 and the best profit is 33.09001000000012
For 2022-08-23, the best sell threshold is 0, th

For 2022-11-23, the best sell threshold is 5, the best buy threshold is 2 and the best profit is 24.820000000000164
For 2022-11-25, the best sell threshold is 7, the best buy threshold is 7 and the best profit is 40.180000000000064
For 2022-11-28, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-11-29, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-11-30, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-12-01, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-12-02, the best sell threshold is 1, the best buy threshold is 0 and the best profit is 0.9099900000001071
For 2022-12-05, the best sell threshold is 0, the best buy threshold is 100 and the best profit is 0.0
For 2022-12-06, the best sell threshold is 1, the best buy threshold is 0 and the best profit is 3.5700299999998606
For 2022-12-07, the b

In [10]:
# for i in range(len(stocks)):
#     best_profit[i]=find_best(stocks[i],budget=budgets[i])[2]

In [11]:
best=sum(best_profit)/len(dates)
res_best_freq = sum(best_freq)/len(dates)
print(f'with the initial budget {budget}, our average profit for 15 days is {best} and trading frequency is {res_best_freq}')

with the initial budget 1000, our average profit for 15 days is 34.71110732323236 and trading frequency is 8.68939393939394
